In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import create_table_leanx as ctl

import values, helpers
import master_data, text_data, sales_doc_data

In [2]:
# master tables
for method in (
     master_data.users,
     master_data.customers_and_vendors, 
     master_data.plants, 
     master_data.materials, 
     master_data.material_support,
     master_data.routes,
     master_data.company_codes
 ):
     table_dict = method()
     for k, v in table_dict.items():
         table_name = k.split('_')[0]
         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
         df = pd.concat([all_cols, pd.DataFrame(v.values())])
         df.to_csv(f'data/OCPM/master/{table_name}.csv', index=False)

In [3]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.doc_types, 
    text_data.organization,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/text/{table_name}.csv', index=False)

In [4]:
MARC = pd.read_csv('data/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/OCPM/master/MAKT.csv')
KNB1 = pd.read_csv('data/OCPM/master/KNB1.csv')
KNA1 = pd.read_csv('data/OCPM/master/KNA1.csv')

all_prices = {}
all_availabilities = {}
all_material_groups = {}

# get price by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_availabilities[nr] = attr['availability']

def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

def get_params():
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    # all_matnrs = MARC[MARC['WERKS'] == 'PL01']['MATNR'].unique()
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))
    # quantities = [random.randint(12, 60)*12 for _ in range(len(matnrs))] # x dozens of everything
    quantity_factor = random.randint(24, 48) if values.om_plants[plant]['high_value'] else random.randint(6, 18)
    quantities = [random.randint(12, 60)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    prices = []
    availabilities = []
    
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]])
        availabilities.append(all_availabilities[matnrs[i]])
    
    #sales_org=random.choice(list(values.om_sales_orgs.keys()))
    sales_org=random.choice(values.om_plants[plant]['sales_orgs'])
    sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    distribution_channel=random.choice(list(values.om_sales_orgs[sales_org]['distribution_channels'].keys()))
    kunnr = random.choice(list(KNB1[KNB1['BUKRS'] == company_code]['KUNNR']))
    customer_name = KNA1[KNA1['KUNNR'] == kunnr]['NAME1'].values[0]
    payment_term = random.choice(list(KNB1[KNB1['KUNNR'] == kunnr]['ZTERM']))
    credit_risk = values.om_customers[customer_name]['credit_risk']
    sales_doc_type=random.choice(list(values.om_sales_doc_types.keys()))

    total = np.sum([prices[i]*quantities[i] for i in range(len(prices))])

    params = {
        'kunnr': kunnr,
        'credit_risk': credit_risk,
        'payment_term': payment_term,
        'company_code': company_code,
        'plant': plant,
        'matnrs': matnrs,
        'quantities': quantities,
        'prices': prices,
        'sales_org': sales_org,
        'sales_office': sales_office,
        'distribution_channel': distribution_channel,
        'availabilities': availabilities,
        'sales_doc_type': sales_doc_type,
        'total': total
    }

    return params

In [5]:
# sales order tables
sales_doc_tables = {
    'VBAK_json': {},
    'VBKD_json': {},
    'VBUK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'EKBE_json': {}, 
    'MSEG_json': {},
    'VBRK_json': {},
    'VBRP_json': {},
    'BKPF_json': {},
    'BSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'JCDS_json': {},
    'VBFA_json': {}
}


    # transition_prob = np.array([
    #     [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Approve Sales Order
    #     [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Generate Delivery Document
    #     [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Pick Items
    #     [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Post Goods Issue
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Send Invoice
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Recieve Delivery Confirmation
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Clear Invoice
        
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00], # Reject Sales Order
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.25], # Set Credit Block
    #     [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Credit Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00,0.00, 0.35], # Set Delivery Block
    #     [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Delivery Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85, 0.00, 0.00,0.00, 0.15], # Set Sales Order Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.05], # Release Sales Order Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85,0.00, 0.15], # Set Customer Billing Block
    #     [0.00, 0.00, 0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.10], # Release Customer Billing Block
    #     [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.01], # Change Payment Term
        
    #     [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 1.00] # Terminate

for i in range(1_000):
    transition_prob = np.array([
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.05], # Approve Sales Order
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.01], # Generate Delivery Document
        [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.05], # Pick Items
        [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.01], # Post Goods Issue
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.05], # Send Invoice
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.01], # Recieve Delivery Confirmation
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 1.00], # Clear Invoice
        
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 1.00], # Reject Sales Order
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.25], # Set Credit Block
        [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.10], # Release Credit Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.35], # Set Delivery Block
        [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.10], # Release Delivery Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00,0.00, 0.00, 0.05], # Set Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.05], # Release Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.95,0.00, 0.00, 0.05], # Set Customer Billing Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.00], # Release Customer Billing Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.01], # Change Payment Term
        [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.00],  # Change Quantity

        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 1.00] # Terminate

    ])

    """
    Steps map
    Starts at:  Create Sales Order
    0: Approve Sales Order
    1: Generate Delivery Document
    2: Pick Items
    3: Post Goods Issue
    4: Send Invoice
    5: Recieve Delivery Confirmation
    6: Clear Invoice

    7: Reject Sales Order
    8: Set Credit Block
    9: Release Credit Block
    10: Set Delivery Block
    11: Release Delivery Block
    12: Set Sales Order Billing Block
    13: Release Sales Order Billing Block
    14: Set Customer Billing Block
    15: Release Customer Billing Block
    16: Change Payment Term
    17: Change Quantity

    18: Terminate Process
    """

    params=get_params()
    latest_date = helpers.generate_random_date(start_date=datetime(2022, 1, 1), end_date=datetime(2023, 12, 31))
    so_created_date = latest_date
    latest_time = helpers.generate_random_time()
    so_created_by = get_user_name(0.6)
    reqested_delivery_date=helpers.add_random_days(min_days=3, max_days=10, current_date=latest_date)

    sd = sales_doc_data.SalesAndDistribution(
        vbeln=f'{str(uuid.uuid4())[-17:]}', 
        params=params,
        start_date=latest_date
    )

    sd.create_sales_order(
        reqested_delivery_date=reqested_delivery_date,
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        erdat=latest_date,
        ernam=so_created_by,
        atime=latest_time
    )
    # latest_date += helpers.UPTO_WEEK()
    latest_time = helpers.add_random_hours(1, latest_time)

    # decide on apprve or reject sales order
    def nex_step():
        if so_created_by != 'BATCH_JOB':
            if random.random() < 0.2:
                return 7 # Reject Sales Oreder for inaccurate information
        # Quantity Change
        if np.average(params['availabilities']) < 0.6:
            if random.random() < 0.1:
                return 17

        return 0 # Approve
    
    step = nex_step()
    while step != 18: # until termination
        if step == 0:
            sd.approve_sales_order(
                usnam=get_user_name(0.7),
                udate=latest_date,
                atime=latest_time
            )
            
            # Set Credit Block
            if params['credit_risk'] > 0.7:
                transition_prob[step][8] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Customer Billing Block
            elif params['credit_risk'] > 0.5:
                transition_prob[step][14] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Change Payment Term
            elif random.random() > 0.95:
                transition_prob[step][16] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)
        elif step == 1:
            sd.generate_delivery_document(
                ernam=get_user_name(0.8), 
                erdat=latest_date,
                planned_delivery_date=latest_date + helpers.UPTO_WEEK(),
                picking_date=None,
                delivery_date=None,
                confirmation_date=None,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(3, latest_time)

            # Delivery blocks
            if np.average(params['availabilities']) < 0.7:
                latest_time = helpers.add_random_hours(1, latest_time)
                transition_prob[step][10] = 0.5
            else:
                latest_time = helpers.add_random_hours(1, latest_time)
        elif step == 2:
            sd.pick_items(
                usnam=get_user_name(0.2), 
                udate=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 3:
            sd.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.3),
                atime=helpers.generate_random_time()
            )

            # Sales Order Billing Block
            if params['total'] > 25_000:
                transition_prob[step][12] = 0.5
                latest_time = helpers.add_random_hours(12, latest_time)
                latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)   
            else:
                latest_time = helpers.add_random_hours(2, latest_time)
                latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
        elif step == 4:
            sd.create_invoice(
                ernam=get_user_name(0.8),
                erdat=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 5:
            sd.delivery_confirmation(
                usnam=get_user_name(0.8), 
                udate=latest_date
            )
            latest_time = helpers.add_random_hours(1, latest_time)
        
            if get_time_consumption(start_date=so_created_date, planned_target_date=reqested_delivery_date, latest_date=latest_date) > 1.25:
                latest_date = helpers.add_random_days(0, 14, latest_date)
            else:
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 6:
            sd.clear_debit_invoice(
                cpudt=latest_date,
                usnam=get_user_name(0.1),
                cleared_date=latest_date + helpers.UPTO_WEEK(),
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        
        elif step == 7:
            sd.reject_sales_order(
                udate=latest_date,
                usnam=get_user_name(0.9), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 8:
            sd.set_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.2)
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 15, latest_date)
        elif step == 9:
            sd.release_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 10:
            sd.set_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.7), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 30, latest_date)
        elif step == 11:
            sd.release_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 12:
            sd.set_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.9), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 13:
            sd.release_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
                blocked_matnrs=[params['matnrs'][0]]
            )
            latest_time = helpers.add_random_hours(4, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 14:
            sd.set_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.7), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 14, latest_date)
        elif step == 15:
            sd.release_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(5, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 16:
            sd.change_payment_term(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)

        elif step == 17:
            old_quantities = params['quantities']
            new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))
            new_quanity_quantities = [old_quantities[j]*(1+random.random()) for j in new_quantity_lines]
            sd.change_quantity( # likely goes to 'Send PO'
                udate=latest_date,
                usnam=get_user_name(0.1),
                line_numbers=new_quantity_lines,
                line_quantities=new_quanity_quantities
            )
            latest_date += helpers.UPTO_WEEK()
        # update transition matrix to sum to 1
        transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
        step = np.random.choice(19, p=transition_prob[step])

    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

<>:65: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:65: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\t.vonluecken\AppData\Local\Temp\ipykernel_15872\3509140111.py:65: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.00]  # Change Quantity
C:\Users\t.vonluecken\AppData\Local\Temp\ipykernel_15872\3509140111.py:65: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,0.00, 0.00, 0.00]  # Change Quantity
C:\Users\t.vonluecken\AppData\Local\Temp\ipykernel_15872\3509140111.py:65: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1.00, 0.00, 0

TypeError: list indices must be integers or slices, not tuple

In [ ]:
sd.tables['VBAP_json']
for k, v in sd.tables['VBAP_json'].items():
    print(sd.tables['VBAP_json'][k]['KWMENG'])

500
380
280
600
450
380
300
570


In [ ]:
for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/OCPM/sales-document/{table_name}.csv', index=False)

In [ ]:
params

{'kunnr': 'b980-17af77c77831',
 'credit_risk': 0.43,
 'payment_term': 'Z030',
 'company_code': 'AE01',
 'plant': 'PL03',
 'matnrs': ['a180-60f593838cb7',
  '9335-6776956ed6a5',
  'ae96-1522c4f35803',
  'b9fb-7226c17ab5f7',
  'a7dd-5b3ff83ea39f',
  'a083-890a152c42d5',
  'ae9e-be6372bac7a1',
  '9716-1c18ab2d5a6c'],
 'quantities': [500, 380, 280, 600, 450, 380, 300, 570],
 'prices': [15.0, 0.4, 0.2, 0.8, 0.5, 6.0, 0.35, 0.7],
 'sales_org': 'EMEA',
 'sales_office': 'GR01',
 'distribution_channel': '20',
 'availabilities': [0.6, 0.85, 0.43, 0.6, 0.8, 0.6, 0.75, 0.2],
 'sales_doc_type': 'ZDIS',
 'total': 11197.0}

In [ ]:
table = 'LIKP_json'
rows = sales_doc_tables[table]
table_name = table.split('_')[0]
all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
all_cols_req = ctl.clean_columns(all_cols,table_name)
df_likp = pd.concat([all_cols_req, pd.DataFrame(rows.values())])


In [ ]:
table = 'CDHDR_json'
rows = sales_doc_tables[table]
table_name = table.split('_')[0]
all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
all_cols_req = ctl.clean_columns(all_cols,table_name)
df_cdhdr = pd.concat([all_cols_req, pd.DataFrame(rows.values())])

In [ ]:
table = 'CDPOS_json'
rows = sales_doc_tables[table]
table_name = table.split('_')[0]
all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
all_cols_req = ctl.clean_columns(all_cols,table_name)
df_cdpos = pd.concat([all_cols_req, pd.DataFrame(rows.values())])

In [ ]:
df_cdpos[(df_cdpos["FNAME"]=='KOSTK') & (df_cdpos["VALUE_NEW"] == 'C')]

,MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,TABNAME,TABKEY,FNAME,CHNGIND,VALUE_NEW,VALUE_OLD
0,SC1,d831497a-e42d-4c9c-8b4a-a83fd37db407,1cc0dc23-11c7-4fd0-87df-33eded9cc68e,9c6e-48e71ab462db,VBUK,SC1ad21-480cdee1a171,KOSTK,U,C,A
17,SC1,65b575a6-6792-4a09-81e8-f64ffe18554e,5ced31c0-c51e-4e15-ab0b-f44ce44f5593,bc6d-9cd5a7bd9355,VBUK,SC1a036-7329f134869d,KOSTK,U,C,A
20,SC1,1d665d30-8fcc-44ce-beaf-1df371137aba,72909bef-a8d0-41d3-a2fa-a5667e82d43a,9a47-33a276bdaf47,VBUK,SC18b50-6055ced36216,KOSTK,U,C,A
24,SC1,25b83880-bbd7-4768-9f75-9e425893ceaa,10cf6792-92cb-4ad9-b071-1b56225ab69e,bb83-accda490c4d3,VBUK,SC181ff-4a850efe5d02,KOSTK,U,C,A
46,SC1,b773a346-21d4-4d5a-8017-74d296c8e75f,97f64f14-72be-4a8d-afd4-d6909caf64cd,87bf-231446a9d0ec,VBUK,SC1a497-76d1b34e7002,KOSTK,U,C,A
...,...,...,...,...,...,...,...,...,...,...
3647,SC1,465f4e59-eb39-4c52-bdec-41cc773b12f3,5bd7c422-9872-47ae-a06e-b701f47c193b,961c-8dda9f975c5c,VBUK,SC1b100-8edec4f9b1af,KOSTK,U,C,A
3649,SC1,99f7fd05-1757-47cf-8d5a-a3d3739baabf,8a2952da-bbe9-4c9b-8d17-879238f289fa,b38c-f2e73314abb0,VBUK,SC1814e-cd0fabb1038c,KOSTK,U,C,A
3651,SC1,d2921827-c548-4944-8d83-4202320d69e6,21ec0a0c-7fff-414e-aaf9-a9a8e9787164,a451-1cd533f6f382,VBUK,SC19f9b-2dd79304109c,KOSTK,U,C,A
3652,SC1,a50ea754-40b1-4961-98f5-9908f0d1194c,3f73ad6a-1dfb-480b-a0fd-5a078cda9fc8,9352-e4b8e0c7e561,VBUK,SC18997-8f19ab3e7356,KOSTK,U,C,A


In [ ]:
df_cdhdr[df_cdhdr[""]]

,MANDANT,OBJECTCLAS,OBJECTID,CHANGENR,USERNAME,UDATE,UTIME,TCODE
0,SC1,d831497a-e42d-4c9c-8b4a-a83fd37db407,1cc0dc23-11c7-4fd0-87df-33eded9cc68e,9c6e-48e71ab462db,Kai Lee,2022-04-10,10:33:14,DEFAULT
1,SC1,73020e4f-de23-446f-8fbf-cb6f1f7df3f5,0e01f6fd-af8c-4424-bb4b-49c328dc5d18,bc29-bddd7aa6f89b,BATCH_JOB,2022-04-23,02:36:32,DEFAULT
2,SC1,bdc147eb-1719-49ca-9294-f4c3d8d0c426,cedc2d5e-ed59-43c1-8c80-6591d15a0100,9fc8-f815c6c3131b,BATCH_JOB,2022-09-10,03:17:48,DEFAULT
3,SC1,e1aa75f4-afa1-4867-a53a-4c6ab9e6cd3e,d26c44e9-9af1-4c81-9ebc-f51309c3d519,95f3-e946595fc98a,BATCH_JOB,2022-09-10,03:26:37,DEFAULT
4,SC1,ecca42c2-5bdd-4ac2-b871-4905c3b86e4d,4b8dbfd9-d9ab-4eb5-9caf-d41874b65be7,90be-49d31b6dea5e,BATCH_JOB,2022-09-10,07:13:22,DEFAULT
...,...,...,...,...,...,...,...,...
3650,SC1,abf5e431-41ad-4b5c-b874-c6e7cc384bc0,f28113b4-f85b-4b27-be53-7c6ad3208568,adf9-7401d498943a,Bailey Smith,2023-07-06,07:54:16,DEFAULT
3651,SC1,d2921827-c548-4944-8d83-4202320d69e6,21ec0a0c-7fff-414e-aaf9-a9a8e9787164,a451-1cd533f6f382,Riley Walker,2022-06-05,06:06:54,DEFAULT
3652,SC1,a50ea754-40b1-4961-98f5-9908f0d1194c,3f73ad6a-1dfb-480b-a0fd-5a078cda9fc8,9352-e4b8e0c7e561,Olive White,2022-10-04,05:23:44,DEFAULT
3653,SC1,687a24f9-4be0-4306-8385-a56e98dd3662,6c7bfae3-1a87-45a6-be45-63bd6de8c4b9,a1b3-6293d8f15d2a,BATCH_JOB,2022-10-17,00:20:04,DEFAULT


In [ ]:
df_likp

,MANDT,VBELN,ERNAM,ERZET,ERDAT,LFART,WADAT,LFDAT,KODAT,VBTYP,KUNNR,BTGEW,NTGEW,GEWEI,VOLUM,VOLEH,KOUHR,PODAT,POTIM
0,SC1,ad21-480cdee1a171,BATCH_JOB,09:25:44,2022-04-10,D,2022-04-16,None,2022-04-10,J,b41e-c86002b71e76,99,594,UNT,99,UNT,10:33:14,2022-04-23,08:42:45
1,SC1,a036-7329f134869d,Olive White,06:28:48,2022-07-24,D,2022-07-26,None,2022-08-06,J,b583-d79c62e38655,99,594,UNT,99,UNT,04:20:53,None,06:28:48
2,SC1,8b50-6055ced36216,BATCH_JOB,11:10:15,2022-11-03,D,2022-11-08,None,2022-11-05,J,bc51-dfa2a2dfa489,99,1782,UNT,99,UNT,09:46:44,2022-11-10,05:13:10
3,SC1,81ff-4a850efe5d02,BATCH_JOB,07:00:41,2022-02-19,D,2022-02-26,None,2022-02-19,J,b41e-c86002b71e76,99,1584,UNT,99,UNT,00:12:55,2022-02-22,07:28:13
4,SC1,a497-76d1b34e7002,BATCH_JOB,03:11:02,2022-09-13,D,2022-09-16,None,2022-09-13,J,b552-7c706428caee,99,1881,UNT,99,UNT,07:10:28,2022-09-26,04:56:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,SC1,b100-8edec4f9b1af,BATCH_JOB,07:32:54,2023-07-15,D,2023-07-18,None,2023-07-15,J,a47a-dd7bd432116e,99,2079,UNT,99,UNT,03:13:01,2023-08-01,00:34:20
831,SC1,814e-cd0fabb1038c,BATCH_JOB,10:59:53,2023-06-24,D,2023-06-27,None,2023-06-24,J,ab72-1c1e9b83f940,99,594,UNT,99,UNT,10:09:23,2023-07-06,06:30:41
832,SC1,9f9b-2dd79304109c,Gabriel Moore,01:12:40,2022-06-05,D,2022-06-07,None,2022-06-05,J,bf6b-ac1820595dd9,99,2475,UNT,99,UNT,06:06:54,None,01:12:40
833,SC1,8997-8f19ab3e7356,BATCH_JOB,05:27:24,2022-10-04,D,2022-10-07,None,2022-10-04,J,b21d-ab2f72f85871,99,1980,UNT,99,UNT,05:23:44,2022-10-17,11:33:41
